In [8]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import bitstring
from qiskit_aer import AerSimulator

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

from qiskit.circuit.library import UnitaryGate

In [9]:
#algorithm for that shit        ,       n=null, e=eins

nn = ["0000","1111"]
ne = ["0011","1100"]
en = ["0101","1010"]
ee = ["0110","1001"]

def idk(new: list, a: list, b:list, c:list):
    for i in a:
        for j in b:
            for k in c:
                new.append(i+j+k)

In [89]:
def code():                                     #intialize |00> state
    qr = QuantumRegister(15,"q")
    cbits = ClassicalRegister(19,"c")
    qc = QuantumCircuit(qr, cbits)

    q = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]

    qc.h(0)
    qc.h(1)
    qc.h(2)
    qc.h(7)
    qc.h(11)
    qc.h(12)

    qc.cx(12,3)
    qc.cx(7,13)
    qc.cx(11,14)


    qc.cx(0,3)
    qc.cx(1,3)
    qc.cx(2,3)

    qc.cx(7,4)
    qc.cx(7,5)
    qc.cx(7,6)

    qc.cx(11,8)
    qc.cx(11,9)
    qc.cx(11,10)

    qc.cx(12,3)
    qc.cx(7,13)
    qc.cx(11,14)

    qc.h(12)
    qc.measure(12,0)
    qc.measure(13,1)
    qc.measure(14,2)

    qc.reset(12)
    qc.reset(13)
    qc.reset(14)
    ####################################################
    for i in range(8):
        qc.cx(i,i+4)
    ####################################################
    qc.h(12)

    qc.cx(12,4)
    qc.cx(7,13)
    qc.cx(12,7)
    qc.cx(4,13)
    qc.cx(12,5)
    qc.cx(6,13)
    qc.cx(12,6)
    qc.cx(5,13)

    qc.h(12)
    qc.measure(12,3)
    qc.measure(13,4)

    qc.reset(12)
    qc.reset(13)
    
    ####################################################
    qc.h(12)

    qc.cx(12,13)
    qc.cx(8,13)
    qc.cx(10,12)
    qc.cx(9,12)
    qc.cx(11,13)
    qc.cx(12,13)

    qc.h(12)

    qc.measure(12,5)
    qc.measure(13,6)

    qc.reset(12)
    qc.reset(13)

    q[5],q[7] = q[7],q[5]
    q[6],q[7] = q[7],q[6]

    q[9],q[10] = q[10],q[9]
    q[10],q[11] = q[11],q[10]
    
    return qc, q

def X_L(qc: QuantumCircuit, q: list, pos: int):
    qc.x(q[3])
    qc.x(q[10])
    if pos == 0:
        qc.x(q[0])
        qc.x(q[11])
    elif pos == 1:
        qc.x(q[1])
        qc.x(q[9])

def Z_L(qc: QuantumCircuit, q: list, pos: int):
    qc.z(q[0])
    qc.z(q[9])
    if pos == 0:
        qc.z(q[3])
        qc.z(q[11])
    elif pos == 1:
        qc.z(q[1])
        qc.z(q[10])

def CNOT_L(qc: QuantumCircuit, q: list, control = 0):
    for i in range(4):
        q[i], q[i+8] = q[i+8], q[i]
    
    if control == 0:
        q[0], q[1] = q[1], q[0]
        q[4], q[5] = q[5], q[4]
        q[8], q[9] = q[9], q[8]
    else:
        q[0], q[2] = q[2], q[0]
        q[4], q[6] = q[6], q[4]
        q[8], q[10] = q[10], q[8]

def H_L(qc: QuantumCircuit, q: list):
    for i in range(12):
        qc.h(i)
    q[0], q[3] = q[3], q[0]
    q[5], q[6] = q[6], q[5]
    q[9], q[10] = q[10], q[9]

def H_L_test(qc: QuantumCircuit, q: list, pos: int):
    anc = qc.num_qubits - 1
    if pos != 2:
        qc.reset(anc)
        
        Z_L(qc, q, pos=pos)

        if pos == 0:                  #hier zu Z_L
            qc.cx(q[0], anc)
            qc.cx(q[3], anc)
            qc.cx(q[9], anc)
            qc.cx(q[11], anc)
        elif pos == 1:
            qc.cx(q[0], anc)
            qc.cx(q[1], anc)
            qc.cx(q[9], anc)
            qc.cx(q[10], anc)
        qc.h(anc)
        if pos == 0:
            qc.cx(anc, q[0])
            qc.cx(anc, q[3])
            qc.cx(anc, q[10])
            qc.cx(anc, q[11])
        elif pos == 1:
            qc.cx(anc, q[1])
            qc.cx(anc, q[3])
            qc.cx(anc, q[9])
            qc.cx(anc, q[10])
        qc.h(anc)
        qc.measure(anc, 7)
        if pos == 0:                        #hier zu X_L
            with qc.if_test((7,1)):
                qc.z(q[0])
                qc.z(q[3])
                qc.z(q[9])
                qc.z(q[11])

                qc.x(q[0])
                qc.x(q[3])
                qc.x(q[10])
                qc.x(q[11])
            #q[9], q[10] = q[10], q[9]
        elif pos == 1:
            with qc.if_test((7,1)):
                qc.z(q[0])
                qc.z(q[1])
                qc.z(q[9])
                qc.z(q[10])

                qc.x(q[1])
                qc.x(q[3])
                qc.x(q[9])
                qc.x(q[10])
            #q[0], q[3] = q[3], q[0]
    if pos == 2:
        for i in range(12):
            qc.h(i)
        q[0], q[3] = q[3], q[0]
        q[5], q[6] = q[6], q[5]
        q[9], q[10] = q[10], q[9]

def S_L(qc: QuantumCircuit, q: list, pos: int):
    anc = qc.num_qubits - 1
    qc.reset(anc)
    qc.h(anc)
    qc.s(anc)

    if pos == 0:
        qc.cx(q[0], anc)
        qc.cx(q[3], anc)
        qc.cx(q[9], anc)
        qc.cx(q[11], anc)
    elif pos == 1:
        qc.cx(q[0], anc)
        qc.cx(q[1], anc)
        qc.cx(q[9], anc)
        qc.cx(q[10], anc)

    qc.measure(anc,7)
    if pos == 0:
        with qc.if_test((7,1)):
            qc.z(q[0])
            qc.z(q[9])
            qc.z(q[3])
            qc.z(q[11])
    elif pos == 1:
        with qc.if_test((7,1)):
            qc.z(q[0])
            qc.z(q[9])
            qc.z(q[1])
            qc.z(q[10])

def readout(qc: QuantumCircuit, shots: int, q: list, noise = 0):
    for i in range(12):
        qc.measure(q[i],18-i)
    
    sim = AerSimulator()
    job = sim.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    nn = ["0000","1111"]
    ne = ["0011","1100"]
    en = ["0101","1010"]
    ee = ["0110","1001"]
    zerozero = []
    zeroone = []
    onezero = []
    oneone = []

    for i in range(1):
        idk(zerozero, nn, nn, nn)
        idk(zerozero, ne, en, ee)
        idk(zerozero, en, ee, ne)
        idk(zerozero, ee, ne, en)
        idk(zeroone, en, nn, ee)
        idk(zeroone, ee, en, nn)
        idk(zeroone, nn, ee, en)
        idk(zeroone, ne, ne, ne)
        idk(onezero, ee, nn, ne)
        idk(onezero, en, en, en)
        idk(onezero, ne, ee, nn)
        idk(onezero, nn, ne, ee)
        idk(oneone, ne, nn, en)
        idk(oneone, nn, en, ne)
        idk(oneone, ee, ee, ee)
        idk(oneone, en, ne, nn)

    bitstring = list(counts.keys())
    hmm = list(counts.values())

    pre, preselected = [i[12:] for i in bitstring], 0
    bits = [i[:12] for i in bitstring]

    for i in range(len(pre)):
        if pre[i].count("1") != 0:
            bits[i] = "pre"
            preselected += hmm[i]
    
    for i in range(len(bits)):
        for j in zerozero:
            if j == bits[i]:
                bits[i] = 0
                break
        if bits[i] != 0:
            for j in zeroone:
                if j == bits[i]:
                    bits[i] = 1
                    break
        if bits[i] != 1 and bits[i] != 0:
            for j in onezero:
                if j == bits[i]:
                    bits[i] = 2
                    break
        if bits[i] != 2 and bits[i] != 1 and bits[i] != 0:
            for j in oneone:
                if j == bits[i]:
                    bits[i] = 3
                    break
        if bits[i] != 0 and bits[i] != 1 and bits[i] != 2 and bits[i] != 3 and bits[i] != "pre":
            bits[i] = "post"

    nullnull = 0
    nulleins = 0
    einsnull = 0
    einseins = 0
    post = 0

    for i in range(len(bits)):
        if bits[i] == 0:
            nullnull += hmm[i]
        if bits[i] == 1:
            nulleins += hmm[i]
        if bits[i] == 2:
            einsnull += hmm[i]
        if bits[i] == 3:
            einseins += hmm[i]
        if bits[i] == "post":
            post += hmm[i]

    nullnull = (nullnull/shots)*100
    nulleins = (nulleins/shots)*100
    einsnull = (einsnull/shots)*100
    einseins = (einseins/shots)*100
    preselected = (preselected/shots)*100
    post = (post/shots)*100

    return [preselected, post, nullnull, nulleins, einsnull, einseins]

def readout_anders_lol(qc: QuantumCircuit, shots: int, q: list, noise = 0):      #einfacher ablesblar, was genau ich messe, aber macht das selbe
    for i in range(12):
        qc.measure(q[i],18-i)
    
    sim = AerSimulator()
    job = sim.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    nn = ["0000","1111"]
    ne = ["0011","1100"]
    en = ["0101","1010"]
    ee = ["0110","1001"]

    zerozero = ["000","123","231","312"]
    zeroone = ["203","320","032","111"]
    onezero = ["301","222","130","013"]
    oneone = ["102","021","333","210"]

    bitstring = list(counts.keys())
    hmm = list(counts.values())

    pre, preselected = [i[12:] for i in bitstring], 0
    bits = [i[:12] for i in bitstring]

    C4_1 = [i[:4] for i in bits]
    C4_2 = [i[4:8] for i in bits]
    C4_3 = [i[8:12] for i in bits]

    test = []

    for i in range(len(pre)):
        if pre[i].count("1") != 0:
            bits[i] = "pre"
            preselected += hmm[i]

    for i in range(len(C4_1)):
        for j in nn:
            if j == C4_1[i]:
                C4_1[i] = "0"
                break
        if C4_1[i] != 0:
            for j in ne:
                if j == C4_1[i]:
                    C4_1[i] = "1"
                    break
        if C4_1[i] != 1 and C4_1[i] != 0:
            for j in en:
                if j == C4_1[i]:
                    C4_1[i] = "2"
                    break
        if C4_1[i] != 2 and C4_1[i] != 1 and C4_1[i] != 0:
            for j in ee:
                if j == C4_1[i]:
                    C4_1[i] = "3"
                    break
        if C4_1[i] != "0" and C4_1[i] != "1" and C4_1[i] != "2" and C4_1[i] != "3":
            bits[i] = "post"

    for i in range(len(C4_2)):
        for j in nn:
            if j == C4_2[i]:
                C4_2[i] = "0"
                break
        if C4_2[i] != 0:
            for j in ne:
                if j == C4_2[i]:
                    C4_2[i] = "1"
                    break
        if C4_2[i] != 1 and C4_2[i] != 0:
            for j in en:
                if j == C4_2[i]:
                    C4_2[i] = "2"
                    break
        if C4_2[i] != 2 and C4_2[i] != 1 and C4_2[i] != 0:
            for j in ee:
                if j == C4_2[i]:
                    C4_2[i] = "3"
                    break
        if C4_2[i] != "0" and C4_2[i] != "1" and C4_2[i] != "2" and C4_2[i] != "3":
            bits[i] = "post"

    for i in range(len(C4_3)):
        for j in nn:
            if j == C4_3[i]:
                C4_3[i] = "0"
                break
        if C4_3[i] != 0:
            for j in ne:
                if j == C4_3[i]:
                    C4_3[i] = "1"
                    break
        if C4_3[i] != 1 and C4_3[i] != 0:
            for j in en:
                if j == C4_3[i]:
                    C4_3[i] = "2"
                    break
        if C4_3[i] != 2 and C4_3[i] != 1 and C4_3[i] != 0:
            for j in ee:
                if j == C4_3[i]:
                    C4_3[i] = "3"
                    break
        if C4_3[i] != "0" and C4_3[i] != "1" and C4_3[i] != "2" and C4_3[i] != "3":
            bits[i] = "post"

    for i in range(len(C4_1)):
        ehm = C4_1[i] + C4_2[i] + C4_3[i]
        test.append(ehm)

    print(test)

    for i in range(len(test)):
        for j in zerozero:
            if j == test[i]:
                bits[i] = 0
        if bits[i] != 0:
            for j in zeroone:
                if j == test[i]:
                    bits[i] = 1
        if bits[i] != 0 and bits[i] != 1:
            for j in onezero:
                if j == test[i]:
                    bits[i] = 2
        if bits[i] != 0 and bits[i] != 1 and bits[i] != 2:
            for j in oneone:
                if j == test[i]:
                    bits[i] = 3
        if bits[i] != 0 and bits[i] != 1 and bits[i] != 2 and bits[i] != 3:
            bits[i] = "post"
    
    nullnull = 0
    nulleins = 0
    einsnull = 0
    einseins = 0
    post = 0

    for i in range(len(bits)):
        if bits[i] == 0:
            nullnull += hmm[i]
        if bits[i] == 1:
            nulleins += hmm[i]
        if bits[i] == 2:
            einsnull += hmm[i]
        if bits[i] == 3:
            einseins += hmm[i]
        if bits[i] == "post":
            post += hmm[i]

    nullnull = (nullnull/shots)*100
    nulleins = (nulleins/shots)*100
    einsnull = (einsnull/shots)*100
    einseins = (einseins/shots)*100
    preselected = (preselected/shots)*100
    post = (post/shots)*100

    return [preselected, post, nullnull, nulleins, einsnull, einseins]

In [95]:
qc, q = code()

# anc = qc.num_qubits - 1

# qc.x(anc)

# qc.cx(anc, q[0])
# qc.cx(anc, q[3])
# qc.cx(anc, q[10])
# qc.cx(anc, q[11])

H_L_test(qc, q, 0)
H_L_test(qc, q, 2)

result = readout(qc, 10000, q, 0)

print("Preselected: ", result[0], "%")
print("Postselected: ", result[1], "%")
print("00: ", result[2], "%")
print("01: ", result[3], "%")
print("10: ", result[4], "%")
print("11: ", result[5], "%")

#qc.draw("mpl")

Preselected:  0.0 %
Postselected:  0.0 %
00:  50.129999999999995 %
01:  49.87 %
10:  0.0 %
11:  0.0 %
